In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import os
import copy
import hist
from coffea import util
import numpy as np
import itertools
import pandas as pd
import uproot3
import mplhep as hep

In [ ]:
dir1 = 'CoffeaOutputsForCombine/Coffea_FirstRun/'
dir2 = 'CoffeaOutputsForCombine/Coffea_SecondRun/'

btagDir = 'MediumBTag/'#'LooseBTag/'
btagType = 'CSVV2/'
yearDir = '2016/'

APVDir = {
    'preVFP': 'APV/',
    'postVFP': 'noAPV/'
}

od = ''
oddir = ''
if btagDir == '':
    od = '_oldANdisc'
    oddir = 'Old2016_MediumBTag/'
    
MR = ''#'_MistagOnly'

# Systematic Correction / Uncertainty Label
## Set the strings accordingly
#### For bTagSyst: '_btagUnc_\<syst\>' where \<syst\> can be either $central$, $up$ or $down$

In [ ]:
Unc = '_btagUnc_central_method2' #_btagUnc_central_method2
UncDir = Unc + '/'

# Load All Data Eras

In [ ]:
from LoadData import JetHT_Unweighted, JetHT_Weighted, Cutflow

JetHT2016_unweighted = JetHT_Unweighted('MediumBTag', 2016)
JetHT2016_weighted = JetHT_Weighted('MediumBTag', 2016, True, Unc, False)

In [ ]:
Cutflow(JetHT2016_weighted)

# Load All MC

In [ ]:
TTbar_unweighted1, TTbar_unweighted2 = {}, {}
TTbar_weighted1, TTbar_weighted2 = {}, {}
QCD_unweighted = {}
QCD_weighted = {}

In [ ]:
contam = '_ttbarContamRemoved'
for vfp in ['preVFP', 'postVFP']:
    TTbar_unwgt_str1 = f'TTbarRes_0l_UL16{vfp}_TTbar_700_1000'
    TTbar_unwgt_str2 = f'TTbarRes_0l_UL16{vfp}_TTbar_1000_Inf'
    TTbar_unweighted1[vfp] = util.load(f'{dir1}TT/{btagDir}{yearDir}{APVDir[vfp]}{TTbar_unwgt_str1}{od}.coffea')
    TTbar_unweighted2[vfp] = util.load(f'{dir1}TT/{btagDir}{yearDir}{APVDir[vfp]}{TTbar_unwgt_str2}{od}.coffea')
    
    QCD_unwgt_str = f'TTbarRes_0l_UL16{vfp}_QCD'
    QCD_unweighted[vfp] = util.load(f'{dir1}QCD/{btagDir}{yearDir}{APVDir[vfp]}{QCD_unwgt_str}{od}.coffea')
    
    TTbar_wgt_str1 = f'{TTbar_unwgt_str1}_weighted{contam}{Unc}{MR}'
    TTbar_wgt_str2 = f'{TTbar_unwgt_str2}_weighted{contam}{Unc}{MR}'
    TTbar_weighted1[vfp] = util.load(f'{dir2}TT/{btagDir}{yearDir}{APVDir[vfp]}{TTbar_wgt_str1}{od}.coffea')
    TTbar_weighted2[vfp] = util.load(f'{dir2}TT/{btagDir}{yearDir}{APVDir[vfp]}{TTbar_wgt_str2}{od}.coffea')
    
    QCD_wgt_str = f'{QCD_unwgt_str}_weighted{contam}{Unc}{MR}'
    QCD_weighted[vfp] = util.load(f'{dir2}QCD/{btagDir}{yearDir}{APVDir[vfp]}{QCD_wgt_str}{od}.coffea')

# Unweighted MC Cutflow

In [ ]:
for name,output in TTbar_unweighted1.items(): 
    print("-------Unweighted TTbar 700 to 1000 " + name + "--------")
    for i,j in output['cutflow'].items():        
        print( '%20s : %12d' % (i,j) )

In [ ]:
for name,output in TTbar_unweighted2.items(): 
    print("-------Unweighted TTbar 1000 to Inf " + name + "--------")
    for i,j in output['cutflow'].items():        
        print( '%20s : %12d' % (i,j) )

# Stuff for Plotting

In [ ]:
import matplotlib.pyplot as plt
import warnings
import re # regular expressions
import HelpfulPlottingDefs as hpd
warnings.filterwarnings("ignore")

# ---- Reiterate categories ---- #
ttagcats = ["AT&Pt", "at", "pret", "0t", "1t", ">=1t", "2t", ">=0t"] 
btagcats = ["0b", "1b", "2b"]
ycats = ['cen', 'fwd']

list_of_cats = [ t+b+y for t,b,y in itertools.product( ttagcats, btagcats, ycats) ]
list_of_bcats = [ b+y for b,y in itertools.product( btagcats, ycats) ]
label_cats_dict = {i: label for i, label in enumerate(list_of_cats)}

In [ ]:
maindirectory = os.getcwd() 
print(maindirectory)

# Luminosities, Cross Sections & Scale Factors

In [ ]:
Nevts2016 = 625441538 # from dasgoclient

Lum2016 = 35920. # pb^-1 from https://twiki.cern.ch/twiki/bin/viewauth/CMS/PdmVAnalysisSummaryTable
# Lum2017 = 41530.
# Lum2018 = 59740.
# Lum     = 137190.

t_BR = 0.6741
ttbar_BR = 0.4544 #PDG 2019
ttbar_xs1 = 831.76 * (0.09210) #pb For ttbar mass from 700 to 1000
ttbar_xs2 = 831.76 * (0.02474) #pb For ttbar mass from 1000 to Inf
toptag_sf = 0.9
toptag_kf = 0.7
qcd_xs = 1370000000.0 #pb From https://cms-gen-dev.cern.ch/xsdb
Lum = {
    'preVFP': 19800.,
    'postVFP': 16128.
}


# =========== SF =========== #
ttbar2016_sf1, ttbar2016_sf2= {}, {}
qcd2016_sf = {}

for vfp in ['preVFP', 'postVFP']:
    ttbar2016_sf1[vfp] = Lum[vfp]*ttbar_xs1*toptag_sf**2*toptag_kf/TTbar_unweighted1[vfp]['cutflow']['sumw']
    ttbar2016_sf2[vfp] = Lum[vfp]*ttbar_xs2*toptag_sf**2*toptag_kf/TTbar_unweighted2[vfp]['cutflow']['sumw']
    qcd2016_sf[vfp] = Lum[vfp]*qcd_xs/QCD_unweighted[vfp]['cutflow']['sumw']

# Bkg. Est. Shape Closure Test

In [ ]:
list_of_hists = ['probept', 'probep']
# list_of_hists = ['probep']
apvdir = '' #APVDir['preVFP']
for name in list_of_hists:
    SaveDirectory = f'{maindirectory}/BkgEstimate/ClosureTest/NormalizedToUnity/{yearDir}{btagDir}{oddir}{btagType}{apvdir}{name}/' # split histograms into subdirectories
    hpd.DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
    for b_y in list_of_bcats:
        plt.rcParams.update({
        'font.size': 14,
        'axes.titlesize': 18,
        'axes.labelsize': 18,
        'xtick.labelsize': 12,
        'ytick.labelsize': 12
        })

        fig, (ax, rax) = plt.subplots(
            nrows=2,
            ncols=1,
            figsize=(10,8),
            gridspec_kw={"height_ratios": (3, 1)},
            sharex=True,
        )
        # ---- initialize data histograms with first era ---- #
        JetHT2016_unwgt_str = 'UL16preVFP_JetHTB_Data'
        Data_hist_modmass_wgt = JetHT2016_weighted['B_preVFP'][name][JetHT2016_unwgt_str, hpd.ConvertLabelToInt(label_cats_dict, 'pret'+b_y), :]
        
        # ---- Add all data together ---- #
        for vfp in ['preVFP', 'postVFP']:
            #---- Define Histograms from Coffea Outputs ----# 
            if vfp == 'preVFP':
                for Era in ['C', 'D', 'E', 'F']: #exclude B because histogram is initialized with B era
                    JetHT2016_unwgt_str = f'UL16{vfp}_JetHT{Era}_Data'
                    # -- For Mass Modified Background Estimate (Non-Top Multi-Jets; NTMJ)-- #
                    Data_hist_modmass_wgt += JetHT2016_weighted[Era+'_'+vfp][name][JetHT2016_unwgt_str, hpd.ConvertLabelToInt(label_cats_dict, 'pret'+b_y), :]
            else:
                for Era in ['F', 'G', 'H']: #exclude B because histogram is initialized with B era
                    JetHT2016_unwgt_str = f'UL16{vfp}_JetHT{Era}_Data'
                    # -- For Mass Modified Background Estimate (Non-Top Multi-Jets; NTMJ)-- #
                    Data_hist_modmass_wgt += JetHT2016_weighted[Era+'_'+vfp][name][JetHT2016_unwgt_str, hpd.ConvertLabelToInt(label_cats_dict, 'pret'+b_y), :]

        

        # -- SM TTbar MC in Signal Region (Contributes to the Background) -- #
        SMTTbar1 = TTbar_unweighted1['preVFP'][name]['UL16preVFP_TTbar_700_1000', hpd.ConvertLabelToInt(label_cats_dict, '2t'+b_y), :]\
                *ttbar2016_sf1['preVFP']\
                + TTbar_unweighted1['postVFP'][name]['UL16postVFP_TTbar_700_1000', hpd.ConvertLabelToInt(label_cats_dict, '2t'+b_y), :]\
                *ttbar2016_sf1['postVFP']
        SMTTbar2 = TTbar_unweighted2['preVFP'][name]['UL16preVFP_TTbar_1000_Inf', hpd.ConvertLabelToInt(label_cats_dict, '2t'+b_y), :]\
                *ttbar2016_sf2['preVFP']\
                + TTbar_unweighted2['postVFP'][name]['UL16postVFP_TTbar_1000_Inf', hpd.ConvertLabelToInt(label_cats_dict, '2t'+b_y), :]\
                *ttbar2016_sf2['postVFP']
        SMTTbar = SMTTbar1 + SMTTbar2

        # -- Mistag Weighted TTbar MC (To Remove Double Counting of Untracked t-tagged J1 Events in Bkg.Est) -- #
        ExtraTTbar1 = TTbar_weighted1['preVFP'][name]['UL16preVFP_TTbar_700_1000', hpd.ConvertLabelToInt(label_cats_dict, 'pret'+b_y), :]\
                   *ttbar2016_sf1['preVFP']\
                   + TTbar_weighted1['postVFP'][name]['UL16postVFP_TTbar_700_1000', hpd.ConvertLabelToInt(label_cats_dict, 'pret'+b_y), :]\
                   *ttbar2016_sf1['postVFP']
        ExtraTTbar2 = TTbar_weighted2['preVFP'][name]['UL16preVFP_TTbar_1000_Inf', hpd.ConvertLabelToInt(label_cats_dict, 'pret'+b_y), :]\
                   *ttbar2016_sf2['preVFP']\
                   + TTbar_weighted2['postVFP'][name]['UL16postVFP_TTbar_1000_Inf', hpd.ConvertLabelToInt(label_cats_dict, 'pret'+b_y), :]\
                   *ttbar2016_sf2['postVFP']
        ExtraTTbar = ExtraTTbar1 + ExtraTTbar2
        

        # =============================================================================================================================== #
        
        # ---- TTbar MC Scaling ---- #
        # SMTTbar *= (ttbar2016_sf) # Prepare to include this with background estimate
        # ExtraTTbar *= (-ttbar2016_sf) # Prepare to subtract this from background to correct for ttbar contamination
        
        # ---- Add SM ttbar MC to background and remove excess events ---- #
        Data_hist_modmass_wgt += (SMTTbar) # Include signal region SM ttbar contribution to NTMJ for the complete background estimate
        #(mistagged ttbar events need to be removed)#
        Data_hist_modmass_wgt += (ExtraTTbar) # ttbar contamination subtraction from background estimate 
        #(removes doubly counted ttbar events included in previous line)#
        

        # =============================================================================================================================== #

        # ---- Background and Observed Signal for Histograms Compared with MC Signals ---- #

        # -- J0 t-tagged and weighted, J1 Mass Modified, SM ttbar included, J1 t-tagged double counts removed -- #
        Background_ModMass_Corrected = Data_hist_modmass_wgt 

        # -- QCD MC -- #
        QCD_hist = QCD_weighted['preVFP'][name]['UL16preVFP_QCD', hpd.ConvertLabelToInt(label_cats_dict, '2t'+b_y), :]\
                 *qcd2016_sf['preVFP']\
                 + QCD_weighted['postVFP'][name]['UL16postVFP_QCD', hpd.ConvertLabelToInt(label_cats_dict, '2t'+b_y), :]\
                 *qcd2016_sf['postVFP']

        # QCD_hist *= qcd2016_sf #scaled according to luminosity 
        QCD_hist += SMTTbar
        
        # ---- Extract both the estimate and QCD events from histograms ---- #
        NtotalQCD = np.sum(QCD_hist.view().value)
        NtotalData = np.sum(Background_ModMass_Corrected.view().value)
        
        # # ---- Normalize QCD histogram directly to the estimate (compare shape) ---- #
        # if NtotalQCD > 0.:
        #     QCD_hist *= (NtotalData/NtotalQCD)
        # else:
        #     QCD_hist *= 0.
        
        # ---- Normalize to Unity (compare shape) ---- #
        if NtotalQCD > 0.:
            QCD_hist *= (1.0/NtotalQCD)
        else:
            QCD_hist *= 0.
        if NtotalData > 0.:
            Background_ModMass_Corrected *= (1.0/NtotalData)
        else:
            Background_ModMass_Corrected *= 0.


        # ---- Plot Histograms (Comment out whatever you don't want included in the figure and/or root file) ---- #
        QCDPlot = QCD_hist.plot1d(ax=ax, histtype='fill', color='yellow')
        BackgroundPlot = Background_ModMass_Corrected.plot1d(ax=ax, histtype='errorbar', marker='.', markersize=5., color='k')
#         TTbarPlot = TTbar_2016_wgt_forplot.plot1d(ax=ax, histtype='fill', color='r')
        
        # plt.ylim(bottom = .1, top = 10**4)

        ax.set_yscale('log')
        # ax.set_ylim(bottom=1.0)
#         ax.autoscale('y')
#         ax.autoscale('x')#, tight=True) # doesn't look like its working...
        ax.set_ylabel('Events')
        ax.set_xlabel(None)
        ax.set_title(name + ' ' + b_y)

        #---- Plot Ratio ----#
        hpd.plotratio(QCD_hist, Background_ModMass_Corrected, ax = rax, histtype='errorbar', marker='.', markersize=4., color='k')
        
        l1 = [r'QCD pythia8 SR', r'Data-Driven Bkg. Est.']
        legtitle1 = ax.legend(labels=l1, fontsize='medium')
        
        
        rax.set_ylabel('Sim./Bkg.')
        rax.axhline(y=1, color='k', linestyle=':')
        rax.set_ylim(0,2)
        
        CMSx, CMSy = 0.01, 0.98 # Position of CMS Preliminary label
        if 'ttbarmass' in name:
            rax.set_xlim(950,6000)
            CMSx = 0.14 
            CMSy = 0.98
        elif 'jetpt' in name:
            rax.set_xlim(400,2000)
            CMSx = 0.26 
            CMSy = 0.98
        elif 'jeteta' in name:
            rax.set_xlim(-2.3,2.3)
        elif 'jetphi' in name:
            rax.set_xlim(-3.14, 3.14)
        elif 'jety' in name:
            rax.set_xlim(-3., 3.)
        elif 'jetdy' in name:
            rax.set_xlim(0., 5.)
        elif 'probept' in name:
            rax.set_xlim(400., 2000.)
            CMSx = 0.14 
            CMSy = 0.98
        elif 'probep' in name:
            rax.set_xlim(400., 7000.)  
            CMSx = 0.14 
            CMSy = 0.98
        

        #---- Labeling ----#
        Lint = str(Lum2016*.001) # Integrated Luminosity
        lumi = plt.text(1.15, 1.07, "L = " + Lint[:6] + " fb$^{-1}$",
                fontsize='x-large',
                horizontalalignment='right',
                verticalalignment='top',
                transform=ax.transAxes
               )
        CMS = plt.text(CMSx, CMSy, 'CMS Preliminary',
                fontsize='x-large',
                horizontalalignment='left',
                verticalalignment='top',
                transform=ax.transAxes
               )
        coffea = plt.text(1.00, 0.85, u"☕",
                  fontsize=50,
                  horizontalalignment='left',
                  verticalalignment='bottom',
                  transform=ax.transAxes
                 )

        filename = 'BkgEst_' + name + '' + Unc + '_' + b_y + '_Closure.png'
        # plt.savefig(SaveDirectory+filename, bbox_inches="tight")
        print('\n' + SaveDirectory + filename)


## Analysis Plots with Data Driven Background Estimate

In [ ]:
""" ---------- Comparing Background Estimate to Unweighted Data (One Histogram) ---------- """
""" --------------- Background: t tagged probe jet from data weighted by mistag --------------- """
""" --------------- Data: Unweighted Data from the Signal Region (2t tag region) -------------- """
apvdir = ''#APVDir['preVFP']
count = 0
DataOutName = f'{maindirectory}/BkgEstimate/{yearDir}{btagDir}{oddir}{btagType}{apvdir}SignalRegionOutput.txt'
filler = 'w'
PlotType = 'log'

# list_of_hists = ['ttbarmass', 'jetpt', 'jeteta', 'jetphi', 'jety', 'jetdy', 'probept', 'probep']
list_of_hists = ['ttbarmass']

for name in list_of_hists:
    SaveDirectory = f'{maindirectory}/BkgEstimate/{yearDir}{btagDir}{oddir}{btagType}{apvdir}{name}/{PlotType}/' # split histograms into subdirectories
    hpd.DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
    for b_y in list_of_bcats:
        plt.rcParams.update({
        'font.size': 14,
        'axes.titlesize': 18,
        'axes.labelsize': 18,
        'xtick.labelsize': 12,
        'ytick.labelsize': 12
        })

        fig, (ax, rax) = plt.subplots(
            nrows=2,
            ncols=1,
            figsize=(10,8),
            gridspec_kw={"height_ratios": (3, 1)},
            sharex=True,
        )
        # ---- initialize data histograms with first era ---- #
        JetHT2016_unwgt_str = 'UL16preVFP_JetHTB_Data'
        Data_hist_unwgt = JetHT2016_unweighted['B_preVFP'][name][JetHT2016_unwgt_str, hpd.ConvertLabelToInt(label_cats_dict, '2t'+b_y), :]
        Data_hist_modmass_wgt = JetHT2016_weighted['B_preVFP'][name][JetHT2016_unwgt_str, hpd.ConvertLabelToInt(label_cats_dict, 'pret'+b_y), :]
        
        # ---- Add all data together ---- #
        for vfp in ['preVFP', 'postVFP']:
            #---- Define Histograms from Coffea Outputs ----# 
            if vfp == 'preVFP':
                for Era in ['C', 'D', 'E', 'F']: #exclude B because histogram is initialized with B era
                    JetHT2016_unwgt_str = f'UL16{vfp}_JetHT{Era}_Data'
                    # -- For Observed Signal -- #
                    Data_hist_unwgt += JetHT2016_unweighted[Era+'_'+vfp][name][JetHT2016_unwgt_str, hpd.ConvertLabelToInt(label_cats_dict, '2t'+b_y), :]
                    # -- For Mass Modified Background Estimate (Non-Top Multi-Jets; NTMJ)-- #
                    Data_hist_modmass_wgt += JetHT2016_weighted[Era+'_'+vfp][name][JetHT2016_unwgt_str, hpd.ConvertLabelToInt(label_cats_dict, 'pret'+b_y), :]
            else:
                for Era in ['F', 'G', 'H']: #exclude B because histogram is initialized with B era
                    JetHT2016_unwgt_str = f'UL16{vfp}_JetHT{Era}_Data'
                    # -- For Observed Signal -- #
                    Data_hist_unwgt += JetHT2016_unweighted[Era+'_'+vfp][name][JetHT2016_unwgt_str, hpd.ConvertLabelToInt(label_cats_dict, '2t'+b_y), :]
                    # -- For Mass Modified Background Estimate (Non-Top Multi-Jets; NTMJ)-- #
                    Data_hist_modmass_wgt += JetHT2016_weighted[Era+'_'+vfp][name][JetHT2016_unwgt_str, hpd.ConvertLabelToInt(label_cats_dict, 'pret'+b_y), :]

        

        # -- SM TTbar MC in Signal Region (Contributes to the Background) -- #
        SMTTbar1 = TTbar_unweighted1['preVFP'][name]['UL16preVFP_TTbar_700_1000', hpd.ConvertLabelToInt(label_cats_dict, '2t'+b_y), :]\
                *ttbar2016_sf1['preVFP']\
                + TTbar_unweighted1['postVFP'][name]['UL16postVFP_TTbar_700_1000', hpd.ConvertLabelToInt(label_cats_dict, '2t'+b_y), :]\
                *ttbar2016_sf1['postVFP']
        SMTTbar2 = TTbar_unweighted2['preVFP'][name]['UL16preVFP_TTbar_1000_Inf', hpd.ConvertLabelToInt(label_cats_dict, '2t'+b_y), :]\
                *ttbar2016_sf2['preVFP']\
                + TTbar_unweighted2['postVFP'][name]['UL16postVFP_TTbar_1000_Inf', hpd.ConvertLabelToInt(label_cats_dict, '2t'+b_y), :]\
                *ttbar2016_sf2['postVFP']
        SMTTbar = SMTTbar1 + SMTTbar2

        # -- Mistag Weighted TTbar MC (To Remove Double Counting of Untracked t-tagged J1 Events in Bkg.Est) -- #
        ExtraTTbar1 = TTbar_weighted1['preVFP'][name]['UL16preVFP_TTbar_700_1000', hpd.ConvertLabelToInt(label_cats_dict, 'pret'+b_y), :]\
                   *ttbar2016_sf1['preVFP']\
                   + TTbar_weighted1['postVFP'][name]['UL16postVFP_TTbar_700_1000', hpd.ConvertLabelToInt(label_cats_dict, 'pret'+b_y), :]\
                   *ttbar2016_sf1['postVFP']
        ExtraTTbar2 = TTbar_weighted2['preVFP'][name]['UL16preVFP_TTbar_1000_Inf', hpd.ConvertLabelToInt(label_cats_dict, 'pret'+b_y), :]\
                   *ttbar2016_sf2['preVFP']\
                   + TTbar_weighted2['postVFP'][name]['UL16postVFP_TTbar_1000_Inf', hpd.ConvertLabelToInt(label_cats_dict, 'pret'+b_y), :]\
                   *ttbar2016_sf2['postVFP']
        ExtraTTbar = ExtraTTbar1 + ExtraTTbar2
        

        # =============================================================================================================================== #
        
        # ---- TTbar MC Scaling ---- #
        # SMTTbar *= (ttbar2016_sf) # Prepare to include this with background estimate
        # ExtraTTbar *= (-ttbar2016_sf) # Prepare to subtract this from background to correct for ttbar contamination
        
        # ---- Add SM ttbar MC to background and remove excess events ---- #
        Data_hist_modmass_wgt += (SMTTbar) # Include signal region SM ttbar contribution to NTMJ for the complete background estimate
        #(mistagged ttbar events need to be removed)#
        Data_hist_modmass_wgt += (ExtraTTbar) # ttbar contamination subtraction from background estimate 
        #(removes doubly counted ttbar events included in previous line)#
        

        # =============================================================================================================================== #

        # ---- Background and Observed Signal for Histograms Compared with MC Signals ---- #

        # -- J0 t-tagged and weighted, J1 Mass Modified, SM ttbar included, J1 t-tagged double counts removed -- #
        Background_ModMass_Corrected = Data_hist_modmass_wgt 

        # -- Simple Observed Data in Signal Region -- #
        Observed = Data_hist_unwgt 
        
        # # ---- Shrink current axis by 10% ---- 3
        # box = ax.get_position()
        # ax.set_position([box.x0, box.y0, box.width * 0.9, box.height * 0.9])

        # ---- Plot Histograms (Comment out whatever you don't want included in the figure and/or root file) ---- #
        ObservedPlot = Observed.plot1d(ax=ax, histtype='errorbar', marker='.', markersize=5., color='k')
        BackgroundPlot = Background_ModMass_Corrected.plot1d(ax=ax, histtype='fill', color='yellow')
        TTbarPlot = SMTTbar.plot1d(ax=ax, histtype='fill', color='r')
        
        NtotalObserved = np.sum(Observed.view().value)
        NtotalBackground = np.sum(Background_ModMass_Corrected.view().value)
        NtotalTTbar = np.sum(SMTTbar.view().value)
        
        
        
        if count > 0:
            filler = 'a'
        if count < 6: # Print number of events for each category once
            with open(DataOutName, filler) as f:
                print(f'\t\t{b_y}\n===================================================', file=f)
                print('TTbar     =', '%10i'% NtotalTTbar,      file=f)
                print('Bkg. Est. =', '%10i'% NtotalBackground, file=f)
                print('Observed  =', '%10i'% NtotalObserved,   file=f)
                print('\n', file=f)
                count += 1
                
        ax.set_yscale(PlotType)
        ax.set_ylim(bottom=1.)
#         ax.autoscale('y')
#         ax.autoscale('x')#, tight=True) # doesn't look like its working...
        ax.set_ylabel('Events')
        ax.set_xlabel(None)
        ax.set_title(name + ' ' + b_y)

        #---- Plot Ratio ----#
        hpd.plotratio(Observed, Background_ModMass_Corrected, ax = rax, histtype='errorbar', marker='.', markersize=4., color='k')
        
        l1 = [r'NTMJ Bkg. Est.', r'SM $t\bar{t}$', r'All Data']
        legtitle1 = ax.legend(labels=l1, fontsize='medium')
        
        
        rax.set_ylabel('Data/Bkg')
        rax.axhline(y=1, color='k', linestyle=':')
        rax.set_ylim(0,2)
        
        CMSx, CMSy = 0.01, 0.98 # Position of CMS Preliminary label
        if 'ttbarmass' in name:
            rax.set_xlim(950,6000)
            CMSx = 0.14 
            CMSy = 0.98
        elif 'jetpt' in name:
            rax.set_xlim(400,2000)
            CMSx = 0.26 
            CMSy = 0.98
        elif 'jeteta' in name:
            rax.set_xlim(-2.3,2.3)
        elif 'jetphi' in name:
            rax.set_xlim(-3.14, 3.14)
        elif 'jety' in name:
            rax.set_xlim(-3., 3.)
        elif 'jetdy' in name:
            rax.set_xlim(0., 5.)
        elif 'probept' in name:
            rax.set_xlim(400., 2000.)
            CMSx = 0.14 
            CMSy = 0.98
        elif 'probep' in name:
            rax.set_xlim(400., 7000.)  
            CMSx = 0.14 
            CMSy = 0.98
        

        #---- Labeling ----#
        Lint = str(Lum2016*.001) # Integrated Luminosity
        lumi = plt.text(1.15, 1.07, "L = " + Lint[:6] + " fb$^{-1}$",
                fontsize='x-large',
                horizontalalignment='right',
                verticalalignment='top',
                transform=ax.transAxes
               )
        CMS = plt.text(CMSx, CMSy, 'CMS Preliminary',
                fontsize='x-large',
                horizontalalignment='left',
                verticalalignment='top',
                transform=ax.transAxes
               )
        coffea = plt.text(1.00, 0.85, u"☕",
                  fontsize=50,
                  horizontalalignment='left',
                  verticalalignment='bottom',
                  transform=ax.transAxes
                 )

        filename = 'BkgEst_' + name + '' + Unc + '_' + b_y + '.png'
        # plt.savefig(SaveDirectory+filename, bbox_inches="tight")
        print('\n' + SaveDirectory + filename)
